# 🏒 Analyseur de Paris LNH 2025-26 - Mise-o-jeu+

Système d'analyse avancé pour optimiser les paris sur la LNH via Mise-o-jeu+ (Loto-Québec).

## 🎯 Objectifs
- Identifier les meilleures **value bets** parmi les cotes actuelles
- Maximiser le rendement avec une approche méthodique
- Équilibrer risque et potentiel de gain

## 📊 Stratégies
- **SAFE**: Favoris solides, faible variance
- **MID**: Valeur cachée, rendement intéressant  
- **BOLD**: Longshots à gros jackpot

In [ ]:
# Imports et configuration
import sys
import os
sys.path.append('/Volumes/Disque Dur/Dev/NHL 2025-2026')

from src.scrapers.news_scraper import NHLNewsScraper
from src.analyzers.value_analyzer import ValueAnalyzer
from src.calculators.odds_calculator import OddsCalculator
from src.calculators.portfolio_manager import PortfolioManager

import pandas as pd
from datetime import datetime

print("🏒 Système d'analyse LNH chargé avec succès!")
print(f"📅 Date d'analyse: {datetime.now().strftime('%d/%m/%Y %H:%M')}")

## 📰 Étape 1: Analyse des Nouvelles Récentes

Surveillance des impacts potentiels sur les cotes:

In [ ]:
# Collecte des nouvelles récentes
news_scraper = NHLNewsScraper()
recent_news = news_scraper.scrape_recent_news(hours_back=24)

print("📰 NOUVELLES RÉCENTES AVEC IMPACT")
print("=" * 40)

for news in recent_news:
    impact_emoji = "🔴" if news['impact'] == 'HIGH' else "🟡"
    print(f"{impact_emoji} [{news['source']}] {news['title']}")
    print(f"   Équipes: {', '.join(news['teams_affected'])}")
    print(f"   Impact: {news['impact']} | {news['summary']}")
    print()

## 🔍 Étape 2: Identification des Opportunités

Recherche des meilleures value bets:

In [ ]:
# Analyse des opportunités de value betting
value_analyzer = ValueAnalyzer()
opportunities = value_analyzer.find_value_bets()

print(f"🎯 {len(opportunities)} opportunités identifiées")
print()

# Créer un DataFrame pour l'analyse
df_opportunities = pd.DataFrame(opportunities)

# Afficher le top 10
print("🏆 TOP 10 des MEILLEURES OPPORTUNITÉS")
print("=" * 50)

top_10 = df_opportunities.head(10)
for idx, row in top_10.iterrows():
    value_pct = row['value'] * 100
    roi_pct = (row['odds'] - 1) * 100
    
    print(f"📊 {row['market']}: {row['selection']}")
    print(f"   Cote: {row['odds']:.2f} | Value: {value_pct:.1f}% | ROI: {roi_pct:.0f}%")
    print(f"   {row['confidence']} | {row['reasoning']}")
    print()

## 💰 Étape 3: Optimisation du Portefeuille

Génération des recommandations selon différents budgets:

In [ ]:
# Test avec différents budgets et stratégies
budgets = [20, 40]  # 20$ ou 2×20$
strategies = ['safe', 'mixed', 'bold']

results = {}

for budget in budgets:
    results[budget] = {}
    
    for strategy in strategies:
        portfolio_manager = PortfolioManager(budget=budget, strategy=strategy)
        recommendations = portfolio_manager.generate_recommendations(opportunities)
        
        total_stake = sum(rec.stake for rec in recommendations)
        total_expected = sum(rec.expected_return for rec in recommendations)
        roi = (total_expected / total_stake * 100) if total_stake > 0 else 0
        
        results[budget][strategy] = {
            'recommendations': recommendations,
            'total_stake': total_stake,
            'expected_return': total_expected,
            'roi': roi
        }

print("📊 COMPARAISON DES STRATÉGIES")
print("=" * 35)

comparison_data = []
for budget in budgets:
    for strategy in strategies:
        result = results[budget][strategy]
        comparison_data.append({
            'Budget': f"{budget}$",
            'Stratégie': strategy.upper(),
            'Misé': f"{result['total_stake']:.2f}$",
            'Retour Attendu': f"{result['expected_return']:.2f}$",
            'ROI': f"{result['roi']:.1f}%"
        })

df_comparison = pd.DataFrame(comparison_data)
print(df_comparison.to_string(index=False))

## 🎯 Recommandations Finales - Budget 20$

Plan de mise optimisé:

In [ ]:
# Recommandations détaillées pour budget 20$ - stratégie mixte
budget = 20
strategy = 'mixed'

portfolio_manager = PortfolioManager(budget=budget, strategy=strategy)
final_recommendations = portfolio_manager.generate_recommendations(opportunities)

print("🎯 PLAN DE MISE RECOMMANDÉ (20$ - MIXTE)")
print("=" * 45)

portfolio_manager.display_recommendations(final_recommendations)

# Analyse des gains potentiels
print("\n📈 ANALYSE DES SCÉNARIOS")
print("=" * 25)

total_stake = sum(rec.stake for rec in final_recommendations)
potential_wins = []

for rec in final_recommendations:
    potential_win = rec.stake * rec.odds
    potential_wins.append(potential_win)
    print(f"Si {rec.selection} gagne: +{potential_win:.2f}$ (mise {rec.stake:.2f}$)")

print(f"\n💰 Total misé: {total_stake:.2f}$")
print(f"🏆 Gain max possible: {max(potential_wins):.2f}$")
print(f"📊 Gain moyen attendu: {sum(rec.expected_return for rec in final_recommendations):.2f}$")

## 🚀 Option Bonus: Scénario "Jackpot"

Longshot à gros potentiel avec 5$ de mise:

In [ ]:
# Identifier le meilleur longshot
longshots = [op for op in opportunities if op['odds'] > 15.0 and op['confidence'] == 'BOLD']

if longshots:
    best_longshot = max(longshots, key=lambda x: x['value'])
    
    print("🎰 OPTION JACKPOT RECOMMANDÉE")
    print("=" * 30)
    print(f"🏒 {best_longshot['market']}: {best_longshot['selection']}")
    print(f"📊 Cote: {best_longshot['odds']:.2f}")
    print(f"💰 Mise suggérée: 5.00$")
    print(f"🏆 Gain potentiel: {5.00 * best_longshot['odds']:.2f}$")
    print(f"🎯 Value: {best_longshot['value']*100:.1f}%")
    print(f"💡 Justification: {best_longshot['reasoning']}")
    
    roi_jackpot = (best_longshot['odds'] - 1) * 100
    print(f"🚀 ROI si succès: {roi_jackpot:.0f}%")
else:
    print("❌ Aucun longshot intéressant identifié")

## 📋 Résumé Exécutif

### Recommandations Finales:

In [ ]:
# Résumé final
print("🏒 RÉSUMÉ EXÉCUTIF - PARIS LNH 2025-26")
print("=" * 45)
print(f"📅 Analyse du: {datetime.now().strftime('%d/%m/%Y')}")
print(f"🎯 Opportunités analysées: {len(opportunities)}")
print(f"📰 Nouvelles considérées: {len(recent_news)}")
print()

print("💡 STRATÉGIE RECOMMANDÉE (Budget 20$):")
safe_count = len([r for r in final_recommendations if r.confidence == 'SAFE'])
mid_count = len([r for r in final_recommendations if r.confidence == 'MID'])
bold_count = len([r for r in final_recommendations if r.confidence == 'BOLD'])

print(f"   • {safe_count} paris SAFE (sécurité)")
print(f"   • {mid_count} paris MID (équilibre)")
print(f"   • {bold_count} paris BOLD (jackpot)")
print()

total_expected = sum(rec.expected_return for rec in final_recommendations)
total_staked = sum(rec.stake for rec in final_recommendations)

print(f"📊 PERFORMANCE ATTENDUE:")
print(f"   • ROI moyen: {(total_expected/total_staked)*100:.1f}%")
print(f"   • Valeur attendue: +{total_expected:.2f}$")
print(f"   • Risque: Modéré (stratégie mixte)")
print()

print("🏆 Bonne chance avec vos paris! 🍀")